# Creating a feature matrix from a DB Query (Opto Sessions)

In [13]:
import sys
sys.path.append('/Users/Sabatini Lab/GitHub/mouse_bandit/data_preprocessing_code/opto_preprocessing')
sys.path.append('/Users/Sabatini Lab/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing_opto as bp
from sklearn import preprocessing
import sys
import os
%matplotlib inline 

## Retrieve names of sessions you want from the DB

In [14]:
#load in data base
db = pd.read_csv('/Users/Sabatini Lab/GitHub/mouse_bandit/mike_session_record.csv',index_col=0)

### Query all 90-10 sessions where performance > 0.5 block structure was 50 and had laser trials @ 10% of trials

In [15]:
r = db[((db['Left Reward Prob'] == 0.9) |  (db['Right Reward Prob'] == 0.9))].copy()
r = r[r['p(high Port)'] > 0.5].copy()
r = r[r['Block Range Min'] == 50].copy()
r = r[r['Laser Stim Prob'] == 0.1].copy()

session_names = r['Session ID'].values 

In [16]:
    r.tail(10)

,Owner,Session ID,Mouse ID,Date,Phase,Laser Stim Prob,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
1138,mike,08112017_SOM44,SOM44,2017-08-11 00:00:00,2,0.1,0.1,0.9,50,50,281,4,214,0.82,2,1.0,40,40
1139,mike,08112017_SOM45,SOM45,2017-08-11 00:00:00,2,0.1,0.1,0.9,50,50,542,9,445,0.90,2,1.0,40,40
1148,mike,08152017_SOM41,SOM41,2017-08-15 00:00:00,2,0.1,0.1,0.9,50,50,533,9,447,0.92,2,1.0,40,40
1149,mike,08152017_SOM44,SOM44,2017-08-15 00:00:00,2,0.1,0.1,0.9,50,50,398,6,327,0.90,2,1.0,40,40
1150,mike,08152017_SOM45,SOM45,2017-08-15 00:00:00,2,0.1,0.1,0.9,50,50,602,9,455,0.86,2,1.0,40,40
1160,mike,08172017_SOM41,SOM41,2017-08-17 00:00:00,2,0.1,0.9,0.1,50,50,492,8,404,0.92,2,1.0,40,40
1161,mike,08172017_SOM44,SOM44,2017-08-17 00:00:00,2,0.1,0.9,0.1,50,50,468,7,382,0.89,2,1.0,40,40
1162,mike,08172017_SOM45,SOM45,2017-08-17 00:00:00,2,0.1,0.1,0.9,50,50,556,9,454,0.89,2,1.0,40,40
1173,mike,08212017_SOM44,SOM44,2017-08-21 00:00:00,2,0.1,0.9,0.1,50,50,262,3,156,0.63,2,1.0,40,40
1174,mike,08212017_SOM45,SOM45,2017-08-21 00:00:00,2,0.1,0.9,0.1,50,50,565,8,406,0.77,2,1.0,40,40


In [17]:
r.shape

(39, 18)

## load in csv files (from running exportTrials.m) added in 'Laser Given' column in trials for new data format following opto changes

In [18]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given','Laser Given']

root_dir = '/Users/Sabatini Lab/GitHub/mouse_bandit/data/mike_data/trial_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

## convert into 1 feature matrix

In [19]:
for i,df in enumerate(trial_df):
    
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    

In [20]:
master_matrix.shape

(16594, 54)

In [21]:
master_matrix['Higher p port'].mean()

0.84355791249849343

In [22]:
master_matrix['Laser'].mean()

0.099855369410630349

## Stephenson et al acquired ~6500 trials per animal

In [23]:
master_matrix['Session ID'].unique()

array(['07122017_PV30', '07122017_SOM42', '07142017_SOM42',
       '07172017_SOM41', '07172017_SOM44', '07172017_SOM45',
       '07182017_SOM42', '07192017_SOM41', '07192017_SOM44',
       '07192017_SOM45', '07202017_SOM42', '07212017_SOM44',
       '07212017_SOM45', '07252017_SOM44', '07252017_SOM45',
       '07282017_SOM44', '07282017_SOM45', '08012017_SOM41',
       '08012017_SOM44', '08032017_SOM41', '08032017_SOM44',
       '08032017_SOM45', '08072017_SOM41', '08072017_SOM44',
       '08072017_SOM45', '08092017_SOM41', '08092017_SOM44',
       '08092017_SOM45', '08112017_SOM41', '08112017_SOM44',
       '08112017_SOM45', '08152017_SOM41', '08152017_SOM44',
       '08152017_SOM45', '08172017_SOM41', '08172017_SOM44',
       '08172017_SOM45', '08212017_SOM44', '08212017_SOM45'], dtype=object)

In [24]:
master_matrix.tail(10)

,Mouse ID,Session ID,Trial,Block Trial,Block Reward,Port Streak,Reward Streak,10_Port,10_Reward,10_ITI,...,1_Reward,1_ITI,1_trialDuration,0_ITI,0_trialDuration,Decision,Switch,Higher p port,Reward,Laser
545,SOM45,08212017_SOM45,556.0,17.0,4.0,6.0,-5.0,0.752,0.0,0.0,...,0.0,0.0,1.359,0.617,2.627,0.0,0.0,0.0,0.0,1.0
546,SOM45,08212017_SOM45,557.0,18.0,4.0,7.0,-6.0,0.530,0.0,0.0,...,0.0,0.0,2.627,0.752,1.288,1.0,1.0,1.0,1.0,0.0
547,SOM45,08212017_SOM45,558.0,19.0,5.0,1.0,1.0,0.588,0.0,0.0,...,1.0,1.0,1.288,0.530,1.169,0.0,1.0,0.0,0.0,0.0
548,SOM45,08212017_SOM45,559.0,20.0,5.0,1.0,-1.0,0.386,1.0,1.0,...,0.0,0.0,1.169,0.588,2.134,0.0,0.0,0.0,0.0,0.0
549,SOM45,08212017_SOM45,560.0,21.0,5.0,2.0,-2.0,0.566,0.0,1.0,...,0.0,0.0,2.134,0.386,1.268,0.0,0.0,0.0,1.0,0.0
550,SOM45,08212017_SOM45,561.0,22.0,6.0,3.0,1.0,0.442,0.0,0.0,...,0.0,1.0,1.268,0.566,1.452,0.0,0.0,0.0,0.0,0.0
551,SOM45,08212017_SOM45,562.0,23.0,6.0,4.0,-1.0,0.519,0.0,0.0,...,0.0,0.0,1.452,0.442,1.149,0.0,0.0,0.0,0.0,0.0
552,SOM45,08212017_SOM45,563.0,24.0,6.0,5.0,-2.0,0.662,0.0,0.0,...,0.0,0.0,1.149,0.519,1.922,0.0,0.0,0.0,0.0,0.0
553,SOM45,08212017_SOM45,564.0,25.0,6.0,6.0,-3.0,0.605,0.0,0.0,...,0.0,0.0,1.922,0.662,1.388,0.0,0.0,0.0,1.0,0.0
554,SOM45,08212017_SOM45,565.0,26.0,7.0,7.0,1.0,0.577,0.0,0.0,...,0.0,1.0,1.388,0.605,2.436,0.0,0.0,0.0,0.0,0.0


## Save combined feature matrix  

In [25]:
master_matrix.to_csv(os.path.join(root_dir,'0.1opto_trials_9010_high.csv'))